In [7]:
!cd /content

In [8]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install ujson

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-fhn9lckh
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-fhn9lckh
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import numpy as np
import torch
import cv2
import json
import datetime
from segment_anything import sam_model_registry, SamPredictor

# Define paths
path = "valid"
image_dir = '/content/drive/MyDrive/plant_disease_detection/dataset/' + path
annotations_save_dir = '/content/drive/MyDrive/plant_disease_detection/dataset/annotations/plant_disease_detection_without_box'
json_file_path = os.path.join(annotations_save_dir, f"coco_panoptic_{path}.json")

# Ensure the annotations save directory exists
os.makedirs(annotations_save_dir, exist_ok=True)

# Load the SAM model
sam_checkpoint = "/content/drive/MyDrive/plant_disease_detection/models/sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda" if torch.cuda.is_available() else "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

# Specify the number of classes to process at a time
no_of_classes = 7

# Initialize or load COCO Panoptic annotation format
if os.path.exists(json_file_path):
    with open(json_file_path, 'r') as json_file:
        coco_output = json.load(json_file)
else:
    coco_output = {
        "info": {
            "description": "Plant Disease Detection Dataset",
            "version": "1.0",
            "year": datetime.datetime.now().year,
            "contributor": "Labeba Tahsin",
            "date_created": datetime.datetime.now().isoformat()
        },
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }

# Collect category information from subfolder names
categories = coco_output.get("categories", [])
category_id_mapping = {category['name']: category['id'] for category in categories}
existing_category_ids = set(category_id_mapping.values())

annotation_id = max([anno['id'] for anno in coco_output.get("annotations", [])], default=0) + 1
processed_classes = len(existing_category_ids)
remaining_classes = []

for subdir in os.listdir(image_dir):
    if os.path.isdir(os.path.join(image_dir, subdir)) and subdir not in category_id_mapping:
        remaining_classes.append(subdir)

if not remaining_classes:
    print("Annotation done for all classes.")
    exit()

# Process only the specified number of classes
remaining_classes = remaining_classes[:no_of_classes]

for category_name in remaining_classes:
    category_id = len(category_id_mapping) + 1
    category_info = {
        "id": category_id,
        "name": category_name,
    }
    categories.append(category_info)
    category_id_mapping[category_name] = category_id

coco_output["categories"] = categories

# Iterate over selected subfolders and images
for category_name in remaining_classes:
    category_id = category_id_mapping[category_name]
    class_dir = os.path.join(image_dir, category_name)

    for image_file in os.listdir(class_dir):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            try:
                # Load the image
                image_path = os.path.join(class_dir, image_file)
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Failed to load image {image_file}")
                    continue
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                height, width, _ = image_rgb.shape

                # Update file name with full path for COCO annotation
                relative_path = os.path.relpath(class_dir, image_dir)
                full_image_path = os.path.join(path, relative_path, image_file)

                # Add image info to COCO
                image_info = {
                    "id": annotation_id,
                    "file_name": full_image_path,
                    "height": height,
                    "width": width
                }
                coco_output["images"].append(image_info)

                # Set the image for prediction
                predictor.set_image(image_rgb)

                # Define central point
                central_point = np.array([width // 2, height // 2])
                input_points = np.vstack([central_point])
                input_labels = np.ones(input_points.shape[0])
                input_box = np.array([8, 8, width - 8, height - 8])

                # Predict masks using the input points
                masks, scores, logits = predictor.predict(
                    point_coords=input_points,
                    point_labels=input_labels,
                    box=input_box,
                    multimask_output=True,
                )

                # Identify the mask with the highest score
                best_mask_idx = np.argmax(scores)
                best_mask = masks[best_mask_idx]
                best_score = scores[best_mask_idx]

                # Prepare annotation for COCO
                segmentation = np.zeros((height, width), dtype=np.uint16)
                segmentation[best_mask] = annotation_id

                annotation_info = {
                    "id": annotation_id,
                    "image_id": annotation_id,
                    "category_id": category_id,
                    "segmentation": segmentation.tolist(),
                    "area": int(np.sum(best_mask)),
                    "iscrowd": 0
                }
                coco_output["annotations"].append(annotation_info)

                annotation_id += 1

                print(f"Processed {image_file} - Best mask score: {best_score:.3f}")
            except Exception as e:
                print(f"Failed to process {image_file}: {str(e)}")

# Save COCO annotations to a file
with open(json_file_path, 'w') as json_file:
    json.dump(coco_output, json_file, indent=None, separators=(',', ':'))

print(f"Processing complete for classes: {', '.join(remaining_classes)}.")


Processed 3a9b39fb-47f0-4a36-b80d-9cea0c9ff58b___GCREC_Bact.Sp 6173.JPG - Best mask score: 0.994
Processed 53f5c516-74f1-41b1-bd89-115568003933___GCREC_Bact.Sp 5704.JPG - Best mask score: 0.979
Processed 21e92c79-7f6e-457f-887f-a6e7ba02494b___GCREC_Bact.Sp 6049.JPG - Best mask score: 0.991
Processed 2c292d57-d2c0-4147-a551-618f1368f72e___GCREC_Bact.Sp 5903.JPG - Best mask score: 1.010
Processed 0cbaca0d-e422-438f-89df-344b493888b8___GCREC_Bact.Sp 5832.JPG - Best mask score: 1.001
Processed 14f9fe9d-517e-4f70-9a4b-97fff9a26e65___GCREC_Bact.Sp 6295.JPG - Best mask score: 0.997
Processed 38ef51e1-c793-4a6c-b17d-e066bcd533c8___GCREC_Bact.Sp 5639.JPG - Best mask score: 0.995
Processed 355edec8-e1bd-4f57-afa3-cbf03c2f06b1___GCREC_Bact.Sp 3353.JPG - Best mask score: 0.984
Processed 42e0e9cd-15aa-4709-b783-1b1a8b306608___GCREC_Bact.Sp 3755.JPG - Best mask score: 1.000
Processed 4c24021e-90e2-4b84-89c2-c904dc8bb0b4___GCREC_Bact.Sp 5706.JPG - Best mask score: 0.996
Processed 07f5c629-92da-49ad-b

In [13]:
#=========================================================
#================== Verify Segmentation ==================
#=========================================================

import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO

# Define paths
path = "valid"
image_dir = '/content/drive/MyDrive/plant_disease_detection/dataset/' + path
annotations_path = '/content/drive/MyDrive/plant_disease_detection/dataset/annotations/plant_disease_detection_without_box/coco_panoptic_' + path + '.json'

# Load COCO annotations
coco = COCO(annotations_path)

# Helper function to display the segmentation mask
def visualize_segmentation(image, segmentation):
    mask = np.array(segmentation, dtype=np.uint16)
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    color_mask[mask > 0] = [255, 0, 0]  # Red color for mask
    overlay = cv2.addWeighted(image, 1.0, color_mask, 0.5, 0)
    return overlay

# Get all image IDs
all_img_ids = coco.getImgIds()

# Get the last 5 image IDs or change as you want to see
img_ids = all_img_ids[-5:]

# Iterate over the first 3 images in the COCO annotations
for image_info in coco.loadImgs(img_ids):
    image_id = image_info['id']
    image_filename = image_info['file_name']
    image_path = '/content/drive/MyDrive/plant_disease_detection/dataset/'+image_filename

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}")
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Load the annotations for the image
    annotation_ids = coco.getAnnIds(imgIds=image_id)
    annotations = coco.loadAnns(annotation_ids)

    # Overlay the segmentation masks on the image
    overlay = image_rgb.copy()
    for ann in annotations:
        segmentation = ann['segmentation']
        overlay = visualize_segmentation(overlay, segmentation)

    # Display the image with segmentation overlay
    plt.figure(figsize=(6, 6))
    plt.imshow(overlay)
    plt.axis('off')
    plt.title(f"Image ID: {image_id} - {image_filename}")
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# #=========================================================
# #================== Verify Segmentation in Single Image ==================
# #=========================================================

# import os
# import json
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from pycocotools.coco import COCO

# # Define paths
# path = "train"
# image_dir = '/content/drive/MyDrive/plant_disease_detection/dataset/' + path
# annotations_path = '/content/drive/MyDrive/plant_disease_detection/dataset/annotations/plant_disease_detection_without_box/coco_panoptic_' + path + '.json'
# image_filename = 'your_image_filename.jpg'  # Replace with your specific image file name

# # Load COCO annotations
# coco = COCO(annotations_path)

# # Helper function to display the segmentation mask
# def visualize_segmentation(image, segmentation):
#     mask = np.array(segmentation, dtype=np.uint8)
#     color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
#     color_mask[mask > 0] = [255, 0, 0]  # Red color for mask
#     overlay = cv2.addWeighted(image, 1.0, color_mask, 0.5, 0)
#     return overlay

# # Find the image in the COCO annotations
# image_id = None
# for img in coco.loadImgs(coco.getImgIds()):
#     if os.path.basename(img['file_name']) == image_filename:
#         image_id = img['id']
#         break

# if image_id is None:
#     print(f"Image {image_filename} not found in annotations.")
# else:
#     # Load the image
#     image_path = os.path.join(image_dir, os.path.relpath(image_filename, '/content/drive/MyDrive/plant_disease_detection/dataset'))
#     image = cv2.imread(image_path)
#     if image is None:
#         print(f"Failed to load image {image_filename}")
#     else:
#         image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         # Load the annotations for the image
#         annotation_ids = coco.getAnnIds(imgIds=image_id)
#         annotations = coco.loadAnns(annotation_ids)

#         # Overlay the segmentation masks on the image
#         overlay = image_rgb.copy()
#         for ann in annotations:
#             segmentation = ann['segmentation']
#             overlay = visualize_segmentation(overlay, segmentation)

#         # Display the image with segmentation overlay
#         plt.figure(figsize=(6, 6))
#         plt.imshow(overlay)
#         plt.axis('off')
#         plt.title(f"Image ID: {image_id} - {image_filename}")
#         plt.show()


In [10]:
import ujson as json
import time

# Specify the path to your JSON file
json_file_path = '/content/drive/MyDrive/plant_disease_detection/dataset/annotations/plant_disease_detection_without_box/coco_panoptic_valid.json'

# Track the start time
start_time = time.time()

# Open and read the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Track the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Display the section
section = data.get('images', {})
print(json.dumps(section[0], indent=4))

# Print the elapsed time
print(f"Time taken to load the file: {elapsed_time:.4f} seconds")


{
    "id": 1,
    "file_name": "valid\/Tomato___Leaf_Mold\/d2422f17-4d15-4416-8c24-fba263d9b3db___Crnl_L.Mold 8810_180deg.JPG",
    "height": 256,
    "width": 256
}
Time taken to load the file: 16.0997 seconds
